by alex_seikin 2023

## Импорты

In [1]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests
# Импортируем функцию log из модуля math:
from math import log
from tqdm import tqdm
import copy

## Загрузка датасета

In [2]:
# Читаем текстовый файл по url-ссылке
try:
    data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text
    # Предобрабатываем текстовый файл
    data = data.split('\n')
    data.remove('')
    data = data + ['[new chapter]'] 
    print('load src: SF')
except:
    data = requests.get("https://raw.githubusercontent.com/immelstorun/ds_public/main/datasets/war_peace_processed.txt").text
    # Предобрабатываем текстовый файл
    data = data.split('\n')
    data.remove('')
    print('load src: DS')


load src: SF


In [3]:
# Образец датасета (первые 10 слов)
print(data[:10])

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее']


## Общее количество слов и количество уникальных слов

In [4]:
# Всего слов в датасете
print(f'Всего слов в датасете: {len(data)}')

# Всего уникальных слов в датасете
word_set = set(data)

# Удаляем из множества слово, символизирующее раздел между главами
word_set.discard('[new chapter]')
print(f'Всего уникальных слов в датасете: {len(word_set)}')

Всего слов в датасете: 300080
Всего уникальных слов в датасете: 38210


## Расчет частоты каждого слова - word_counts

In [5]:
# Инициализация пустого словаря
word_counts = {}

# Проходим по каждому слову из общего списка слов 
for word in data:
    # Если слова нет в словаре, то добавляем его как ключ, присваиваем значение 1
    if word not in word_counts:
        word_counts[word] = 1
    # Иначе этому ключу увеличиваем значение на 1
    else:
        word_counts[word] += 1
        
# Check
print(f'Всего в словаре: \t\t\t{len(word_counts)} записей')

chapter_count = data.count('[new chapter]')
print(f"Количество вхождений '[new chapter]': \t{chapter_count}")
# Создаем цикл по ключам и их порядковым номерам полученного словаря
print('\nПример словаря:')
for i, key in enumerate(word_counts):
    # Выводим только первые 5 слов
    if i == 5:
        break
    print(key, '\t\t', word_counts[key])

Всего в словаре: 			38211 записей
Количество вхождений '[new chapter]': 	171

Пример словаря:
1 		 10
в 		 6997
два 		 215
раза 		 35
короче 		 3


## Разделение слов по главам - word_by_chapters

In [6]:
# Создание списка списков (в каждом списке слова из отдельной главы)

# Инициализация пустого списка
word_by_chapters = []
# Инициализация временного хранилища слов из главы
temp_chapter = []

for word in data:
    if word == "[new chapter]":
        word_by_chapters.append(temp_chapter)  # Если слово [new chapter] - в список списков добавляется список слов из главы
        temp_chapter = []
    else:
        temp_chapter.append(word)  # Набор слов главы в список до стоп слова [new chapter]

print(f'Всего списков слов: \t{len(word_by_chapters)}')
for i, chapter in enumerate(word_by_chapters[:5], 1):
    word_count = len(chapter)
    print(f'Слов в главе {i}: \t{word_count}')
        

Всего списков слов: 	171
Слов в главе 1: 	643
Слов в главе 2: 	521
Слов в главе 3: 	1
Слов в главе 4: 	2220
Слов в главе 5: 	1060


## Создание списка словарей - freq_by_chapt_dict

In [7]:
# Инициализация пустого списка словарей
freq_by_chapt_dict = []  

for chapter in word_by_chapters:
    word_freq = {} # Каждый словарь - частота втречаемости слова в данной главе
    for word in chapter:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    freq_by_chapt_dict.append(word_freq)

print(f'Всего словарей в списке: \t{len(freq_by_chapt_dict)}')
for i, word_freq in enumerate(freq_by_chapt_dict[:5], 1):
    entry_count = len(word_freq)
    print(f'\nСловарь главы {i}: \t{entry_count} записей')
    print(f'Образец первых 5 записей в словаре: \t{list(word_freq.items())[:5]}')

Всего словарей в списке: 	171

Словарь главы 1: 	407 записей
Образец первых 5 записей в словаре: 	[('1', 1), ('в', 37), ('два', 3), ('раза', 1), ('короче', 1)]

Словарь главы 2: 	298 записей
Образец первых 5 записей в словаре: 	[('автора', 1), ('я', 20), ('пишу', 2), ('до', 1), ('сих', 1)]

Словарь главы 3: 	1 записей
Образец первых 5 записей в словаре: 	[('первая', 1)]

Словарь главы 4: 	1079 записей
Образец первых 5 записей в словаре: 	[('--', 81), ('ну', 5), ('что', 44), ('князь', 21), ('генуя', 1)]

Словарь главы 5: 	633 записей
Образец первых 5 записей в словаре: 	[('гостиная', 1), ('анны', 2), ('павловны', 2), ('начала', 1), ('понемногу', 1)]


## Обзор переменных

* `word_set` - множество из всех слов, которые есть в книге

* `chapter_count` - количество глав в книге (171) ! Количество вхождений '[new chapter] в общий список слов

* `word_counts` - словарь, ключами которого являются слова, а значениями - количество вхождений этих слов в книгу

* `word_by_chapters` - список из 171 списка, где элементы вложенных списков - все слова из главы. Каждый список соответствует своей главе

* `freq_by_chapt_dict` - список из 171 словаря, где ключи - слова, а значения - количество слов в главе. Каждый словарь соответствует своей главе

# Задание 1 - tf

Формула для вычисления `term frequency` для слова `word`:
$$ tf_{word, chapter} = \frac {n_{word, chapter}} {n_{chapter}}$$

где
* ${n_{word, chapter}}$ - сколько раз слово `word` встрачается в главе `chapter`,
* $n_{chapter}$ - количество слов в главе `chapter`.

### Вычисление tf для целевого слова и целевой главы из текста.

In [8]:
target_word = 'два' # Целевое слово
target_chapter = 1 # Целевая глава

if target_chapter - 1 < len(freq_by_chapt_dict):
    word_freq = freq_by_chapt_dict[target_chapter - 1]
    if target_word in word_freq:
        frequency = word_freq[target_word]
        total_entries = len(word_freq)
        tf = frequency / total_entries
        print(f"tf для слова '{target_word}' в главе {target_chapter}: {tf}")
    else:
        print(f'Слова \'{target_word}\' нет в этой главе')
else:
    print('Главы с номером {target_chapter} нет книге')

tf для слова 'два' в главе 1: 0.007371007371007371


### Расчёт tf для всех слов и каждой главы. Результаты сохраняются в виде словаря - tf_result_list.

In [9]:
tf_result_list = []

for chapter in freq_by_chapt_dict: # Для каждого словаря (частота встречаемости слова) из списка словарей
    total_entries = len(chapter) # Определяем длинну словаря
    tf_result_dict = {} # Создаем пустой словарь для результата
    
    for term, frequency in chapter.items(): # Для каждого слова и его количества в главе
        term_frequency = round((frequency / total_entries), 5) # Определяем значение tf
        tf_result_dict[term] = term_frequency # Записываем в новый словарь значение tf для каждого слова
    
    tf_result_list.append(tf_result_dict) # Добавляем словарь статистики tf по главе в общий список словарей

# Check для 1 главы первых 5 слов
for term, tf in list(tf_result_list[0].items())[:5]:
    print(f'Слово: {term} \ttf: {tf}')

Слово: 1 	tf: 0.00246
Слово: в 	tf: 0.09091
Слово: два 	tf: 0.00737
Слово: раза 	tf: 0.00246
Слово: короче 	tf: 0.00246


# Задание 2 - df

Вычисляется по формуле:

$df_{word} = \frac{N_{word}}{N}$,

где
* $ N_{word} $ - число документов (глав) содержащих слово `word`,
* $ N $ - общее число документов (глав).

### Вычисление df для целевого слова из текста.

In [10]:
target_word = 'она'
cnt_target_word = 0
df_dict = {}

# Перебираем каждую главу в списке word_by_chapters
for chapter in word_by_chapters: # Если целевое слово есть в главе, увеличиваем счетчик
    if target_word in chapter:
        cnt_target_word += 1
# Рассчитываем df для целевого слова и сохраняем в словаре df_dict
if cnt_target_word > 0:
    df_dict[target_word] = round((cnt_target_word / len(word_by_chapters)), 5)
else:
    df_dict[target_word] = 0.0

# Выводим словарь df_dict
print(f'df для слова: {df_dict}, встречаемость в {cnt_target_word} главах')

df для слова: {'она': 0.66082}, встречаемость в 113 главах


### Расчёт df для всех уникальных слов из текста. Результаты сохраняются в виде словаря - df_result_dict.

In [11]:
# Инициализируем пустой словарь
df_result_dict = {}
# Преобразуем множество в список
total_chapters = len(word_by_chapters)

# Перебираем первые 1000 слов из word_set
for word in tqdm(list(word_set)):
    df_dict = {}
    cnt = 0

    # Подсчитываем количество раз, когда слово встречается в каждой главе
    for chapter in word_by_chapters:
        if word in chapter:
            cnt += 1

    # Сохраняем относительную частоту встречаемости слова в df_dict
    df_dict[word] = cnt / total_chapters

    # Обновляем df_result_dict с результатами для каждого слова
    df_result_dict.update(df_dict)

# Выводим результаты
print(df_result_dict)

100%|██████████| 38210/38210 [04:42<00:00, 135.42it/s]


{'любимец': 0.005847953216374269, 'отдельный': 0.005847953216374269, 'громадное': 0.005847953216374269, 'провожаемый': 0.011695906432748537, 'пагодами': 0.005847953216374269, 'работавших': 0.005847953216374269, 'освещенные': 0.005847953216374269, 'учиться': 0.04093567251461988, 'удрали': 0.005847953216374269, 'наделал': 0.017543859649122806, 'лазу': 0.005847953216374269, 'передвинуться': 0.005847953216374269, 'свободы': 0.05847953216374269, 'научишься': 0.005847953216374269, 'радовали': 0.011695906432748537, 'обмахивал': 0.005847953216374269, 'подъезжающих': 0.005847953216374269, 'сознанием': 0.017543859649122806, 'смешливого': 0.005847953216374269, 'оглядывали': 0.011695906432748537, '30': 0.023391812865497075, 'главным': 0.029239766081871343, 'весельчак': 0.011695906432748537, 'разорится': 0.005847953216374269, 'всезнающему': 0.005847953216374269, 'палатка': 0.011695906432748537, 'переживает': 0.005847953216374269, 'потребности': 0.07017543859649122, 'обманщики': 0.005847953216374269

# Задание 3 - tf-idf

Формула для вычисления следующая:

`tf-idf = term frequency * inverse document frequency`

* `tf` — это частотность термина, которая измеряет, насколько часто термин встречается в документе.

* `idf` — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.


$$tf\_idf_{word, chapter} = tf * log(\frac{1}{df})

### Вычисление tf-idf для целевого слова и целевой главы из текста. 

In [12]:
target_word = 'она'
target_chapter = 1

tf_target = tf_result_list[target_chapter-1][target_word]
df_target = df_result_dict[target_word]
tf_idf_target = tf_target * log(1/df_target)
print(f'Для слова: \'{target_word}\' из главы: {target_chapter} значение tf_idf: {round(tf_idf_target, 10)}')

Для слова: 'она' из главы: 1 значение tf_idf: 0.0020340939


### Расчёт tf-idf выполнен для всех слов из каждой главы. Результаты сохраняются в виде словаря.

In [14]:
tf_idf_result_list = copy.deepcopy(tf_result_list) # Делаем полное копирование списка словарей с результатами tf

for i in tqdm(range(len(tf_idf_result_list))): # Для общего количества глав
    for word in tf_idf_result_list[i].keys(): # Для каждого ключа, являющегося словом из словаря главы с данными tf
        tf_idf_result_list[i][word] = tf_idf_result_list[i][word] * log(1 / df_result_dict[word]) # Делаем расчет tf-idf для каждого слова и перезаписываем этим значением значение tf
        


100%|██████████| 171/171 [00:00<00:00, 940.72it/s]


In [16]:
# Check
tf_idf_result_list[0]['она']

0.0020340938725504675

# Задание 4

### Поиск трёх слов с наибольшим значением tf-idf для заданной главы.

In [71]:
# Проходимся по первым пяти словарям в tf_idf_result_list и отслеживаем индекс с помощью enumerate
for i, my_dict in enumerate(tf_idf_result_list[:4]):
    # Сортируем элементы словаря по их значениям в порядке убывания
    sorted_items = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Выбираем три самых больших значения из отсортированного списка
    top_three = sorted_items[:5]
    
    # Выводим номер главы для контекста
    print(f"Топ-5 значений tf-idf для главы: {i+1}")
    
    # Проходимся по трем самым большим значениям и отслеживаем индекс с помощью enumerate
    for key, (word, value) in enumerate(top_three):
        # Округляем значение до 5 знаков после запятой
        value_rounded = round(value, 5)
        # Выводим округленное значение
        print(f'{key+1}\t{word}:   \t{value_rounded}')
    
    # Выводим пустую строку для разделения между главами
    print()

Топ-5 значений tf-idf для главы: 1
1	всё:   	0.05497
2	романа:   	0.05467
3	толстой:   	0.04938
4	издания:   	0.03789
5	1866:   	0.03789

Топ-5 значений tf-idf для главы: 2
1	аристократ:   	0.07465
2	думает:   	0.04084
3	купцов:   	0.0345
4	семинаристов:   	0.0345
5	пружин:   	0.0345

Топ-5 значений tf-idf для главы: 3
1	первая:   	2.14593

Топ-5 значений tf-idf для главы: 4
1	павловна:   	0.03016
2	анна:   	0.02019
3	функе:   	0.01429
4	буонапарте:   	0.0123
5	верю:   	0.01186

